In [ ]:
from ipywidgets import interact
import numpy as np
import pandas as pd
import csv
from IPython.display import Image


import bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models.glyphs import Patch
output_notebook()

# Modèle d'atmosphère

On considère un modèle d'atmosphère dans lequel la première couche (troposphère) est une atmosphère adiabatique et la deuxième couche (stratosphère) une atmosphère isotherme.

Au sol, on a $p_0=10^5\;$Pa et une densité de l'air $\rho_0=1.23\;$kg.m$^{-3}$. L'air est un gaz parfait de coefficient $\gamma =1.4$. 

La transition se fait à $h_\text{trans}$, choisie. 

La pression et la masse volumique varient avec l'altitude $h$ selon  $d p = -\rho g\,d h$.

La loi des gaz parfait se réécrit
    
$$T=\frac{pM}{R\rho}$$

La portion adiabatique impose

$$     p\rho^{-\gamma} = \text{cte}=p_0\rho_0^{-\gamma}\quad\Longrightarrow\quad \rho=\rho_0
        \left(\frac{p}{p_0}\right)^{\frac{1}{\gamma}}$$
        
$$       d p = -\rho_0 g \left(\frac{p}{p_0}\right)^{1/\gamma} d h$$

$$      p_\text{tropo}(h) = p_0 \left(1 - \frac{\gamma - 1}{\gamma}\frac{\rho_0g}{p_0}h\right)^\frac{\gamma}{\gamma - 1} = p_0 \left(1-\frac{h}{h_0}\right)^\frac{\gamma}{\gamma - 1}$$
     
$$      h_0 = \frac{\gamma}{\gamma - 1}\frac{p_0}{\rho_0 g}$$    

Et dans la troposphère, on trouve aussi 

$$T_\text{tropo}=T_0\left(1-\frac{h}{h_0}\right)$$

In [ ]:
#Constante des gaz parfaite et données au sol
R=8.314
p0=10**5
rho0=1.23
gamma=1.4
T0=300
g=9.81

In [ ]:
h0=(gamma/(gamma-1))*p0/(g*rho0)

In [ ]:
h0/1000 # h0 en km

In [ ]:
def ptropo(h):
    ptro=p0*(1-h/h0)**(gamma/(gamma-1))
    return(ptro)

def Ttropo(h,T0):
    ttro = T0*(1-h/h0)
    return(ttro)

In [ ]:
h1 = np.linspace(0,29,291)

In [ ]:
f=figure(title="évolution de la pression dans le modèle adiabatique",x_axis_label="altitude (km)",
         y_axis_label="pression (bar)", height=400, width=600)
f.line(h1,ptropo(h1*1000)/10**5,color='darkcyan',line_width=4)

show(f)

In [ ]:
f=figure(title="évolution de la température dans le modèle adiabatique",x_axis_label="altitude (km)",
         y_axis_label="Température (K)", height=400, width=600)
f.line(h1,Ttropo(h1*1000,300),color='darkcyan',line_width=4)

show(f)

### Portion isotherme

Dans la portion isotherme, 

$$ d p=-\frac{Mpg}{RT_\text{strat}}d h$$

$$   \ln(p) = -\frac{Mg}{RT_\text{strat}}h + A$$

$$p_\text{strat} = B e^{\left( -\frac{Mg}{RT_\text{strat}}h\right)}$$
      
Le modèle s'applique à partir de $h_\text{trans}$, transition entre la troposphère et la stratosphère (entre 8 et 18 km suivant la lattitude). 

La pression et la températures sont continues, on obtient $p_\text{trans} = p_\text{tropo} (h_\text{trans})$, et $T_\text{strat} = T_\text{tropo} (h_\text{trans})$

$$\frac{M}{R} = \frac{T \rho}{p} =\frac{T_0\rho_0}{p_0}$$

$$p_\text{strat} = B e^{\left( -\frac{T_0\rho_0 g}{p_0 T_\text{strat}}h\right)}$$


In [ ]:
def pstrat(h):
    pstr= B * np.e**(-(T0*rho0*g*h)/(p0*Tstrat))
    return(pstr)

### Comparaison du modèle adiabatique et isotherme depuis le sol 

In [ ]:
htrans = 0
ptrans = ptropo(htrans)
Tstrat = Ttropo(htrans, 300)
B = ptrans*np.e**(T0*rho0 *g *htrans/(p0*Tstrat))

In [ ]:
f=figure(title="évolution de la pression",x_axis_label="altitude (km)",
         y_axis_label="pression (bar)", height=400, width=600)
f.line(h1,ptropo(h1*1000)/10**5,color='darkcyan',line_width=4, legend_label="modèle adiabatique")
f.line(h1,pstrat(h1*1000)/10**5,color='darkorange',line_width=4, legend_label="modèle isotherme")

show(f)

### transition adiabatique -> isotherme à htrans

In [ ]:
T0=300
htrans = 8000
ptrans = ptropo(htrans)
Tstrat = Ttropo(htrans, T0)
B = ptrans*np.e**(T0*rho0 *g *htrans/(p0*Tstrat))

h2 = np.array([h1i for h1i in h1 if h1i <= htrans/1000])
h3 = np.array([h1i for h1i in h1 if h1i >= htrans/1000])

f=figure(title="évolution de la pression avec l'altitude",x_axis_label="altitude (km)",
         y_axis_label="pression (bar)", height=400, width=600)
f.line(h1,ptropo(h1*1000)/10**5,color='darkcyan',line_width=1, line_dash=[10, 5])
f.line(h2,ptropo(h2*1000)/10**5,color='darkcyan',line_width=4, legend_label="modèle adiabatique")
f.line(h1,pstrat(h1*1000)/10**5,color='darkorange',line_width=1, line_dash=[10, 5])
f.line(h3,pstrat(h3*1000)/10**5,color='darkorange',line_width=4, legend_label="modèle isotherme")

show(f)

print('-')
print('Altitude choisie pour la transition troposphère->stratospère h_trans = {:0.1f}[km]'.format(htrans/1000))
print('Température au sol T0 = {:0.1f}[°C]'.format(T0-273.15))
print('Température de la stratosphère Tstrat = {:0.1f}[°C]'.format(Tstrat-273.15))

## Conclusion
Le modèle présenté ici est très simplifié. Il permet d'expliquer la baisse de pression et de température avec l'altitude, observée lorsqu'on se promène en montagne. La température obtenue pour la stratosphère semble, elle un peu (!) basse. Il manque clairement des éléments au modèle. 

Notebook by Cécile Hébert (2023-2024).
Except where otherwise noted, the content of this notebook is licensed under MIT licence.